Mounting Drive

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory IMPORTANT!
%cd /content/drive/My Drive/DS2500 Project

[Errno 2] No such file or directory: '/content/drive/My Drive/DS2500 Project'
/content


In [ ]:
%pwd

'/content'

Importing Data

In [ ]:
import pandas as pd
import re
from pathlib import Path
import wordcloud as wc
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import imageio
from collections import Counter

pd.set_option('display.max_columns', None)

In [ ]:
def read_file(filename, column1, column2):
  y = '/content/drive/My Drive/DS2500 Project/'+ filename
  x = pd.read_csv(r)
  x[column1] = [word.lower() for word in x[column1]]
  x[column2] = [word.lower() for word in x[column2]]
  x[column1] = [re.sub('[^\w\s]','',word) for word in x[column1]]
  x[column2] = [re.sub('[^\w\s]','',word) for word in x[column2]]
  x['key'] = x[column1] + " " + x[column2]

  return x

In [ ]:
songs = read_file('top_40.csv', 'TITLE', 'ARTIST')

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
song_data = read_file('data.csv', 'name', 'artists')

In [ ]:
combined = songs.merge(song_data, left_on=songs['key'], right_on=song_data['key'],how = 'left')


In [ ]:
combined.columns

In [ ]:
combined.drop_duplicates(subset ="key_0", 
                     keep = "last", inplace = True)
print(combined)

PART 1: ANALYZING SONG INFORMATION

In [ ]:
#visualizing the distribution of duration
import matplotlib.pyplot as plt

def duration(df, column, bins = 10):
  df["duration_second"] = df[column] * 0.001
  plt.hist(df["duration_second"])
  plt.xlabel("Duration of songs in seconds")
  plt.ylabel("Count of songs")
  plt.title("Distribution of Top Songs' Duration")

duration(combined, "duration_ms")

In [ ]:
def ranking(df, column):
    
    df[column] = df[column].astype(int)
    rank = 1
    last_value = df[column[0]]
    ranked_dict = dict()
    for name in column:
      value = df[name]
    if value_x != value_y:
        rank += 1
    ranked_dict[name] = rank
    value_y = value_x

    df1 = pd.DataFrame(ranked_dict.nlargest(40, column))  # Dataframe of songs
    df1[column_name] = [40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]   # Assign score to songs
    
    return df

test = ranking(combined, 'duration_second')
print(test)





In [ ]:
def score(lst):
    mydict = {}
    
    for x in lst:
        
        for song in x['TITLE']:
    
            score= x.loc[x['TITLE'] == team, 'Score'].iloc[0]   #Give individual score to song from previous function
            if song in mydict:   # Find cumulative score for each song
                
                mydict[song] += score
            else:
                
                mydict[song] = score
                
                
                
    return mydict 

PART 2: READING AND ANALYZING LYRICS


In [ ]:
#reads lyric files

def read_lyrics(filename):
    """ Opens and reads file and returns a list of strings """
    with open(filename) as infile:
        lyrics = infile.readlines()
        return lyrics

In [ ]:
roses = read_lyrics('roses.txt')
roses

In [ ]:
stop_words =["a", "an", "and", "the", "to", "i", "if", "of", "that", "it",
            "is", "im", "has", "was", "his", "ive", "at", "in", "your", "its",
            "for", "this", "we", "but", "by", "are", "our", "as"]

In [ ]:
#cleans up lyrics

def clean_text(lyrics):
    ''' Inputs a list of strings, removes punctuation, stop words,
    new line characters, and turns words lower case as a list of strings
    '''
    clean_lyrics = []
    for line in lyrics:
        line = re.sub("[^\w\s]", "", line)
        line = line.lower()
        line = line.replace("\n", "")
        
        clean_line = [word for word in line.split(" ") if word not in stop_words]
             
        if clean_line != [""]:
            clean_string = " ".join(clean_line)
            clean_lyrics.append(clean_string)
            
    return clean_lyrics

In [ ]:
clean_roses = clean_text(roses)

In [ ]:
#word cloud generator

def cloud_generator(filename, image=None):
  lyrics = Path(filename).read_text()
  #MASK = imageio.imread(image)
  generate_cloud = wc.WordCloud(width=1000, height=1000)

  cloud = generate_cloud.generate(lyrics)
  plt.imshow(cloud)


In [ ]:
cloud_generator('roses.txt')

In [ ]:
#scores lyrics for sentiment analysis

def score_songs(lyrics, minsub=0.0, maxsub=1.0, minpol=-1.0, maxpol=1.0):
    """ Score each poem for sentiment (polarity/subjectivity) and return as a dictionary (tweet --> (pol/subj)) """
    
    filtered = {}
    for line in lyrics:
        pol, sub = TextBlob(line).sentiment
        if minpol <= pol <= maxpol and minsub <= sub <= maxsub:
            filtered[line] = (pol, sub)
    
    return filtered

In [ ]:
roses_scored = score_songs(clean_roses)

In [ ]:
#generates sentiment analysis graph

fig, ax = plt.subplots(figsize=(10,10), dpi=100)
plt.xlabel('Subjectivity')
plt.ylabel('Polarity')
plt.title('Top 40 Songs Sentiment Analysis')

def polarity_vs_subjectivity(scored_lyrics, DATA_COLOR, LINE_COLOR, LABEL):
    scores = scored_lyrics.values()
    polarity = [x[0] for x in scores]
    subjectivity = [x[1] for x in scores]
    
    sns.scatterplot(x=subjectivity, y=polarity, s=3, color=DATA_COLOR)
    sns.kdeplot(x=subjectivity, y=polarity, color=LINE_COLOR, label=LABEL)

polarity_vs_subjectivity(roses_scored, 'palegoldenrod', 'palegoldenrod', 'Roses -- SAINt JHN')
plt.legend()

In [ ]:
'''
sns.set(font_scale=.5)
    sns.heatmap(total_df, xticklabels=True, yticklabels=True)
    plt.title("Obama's Top 100 Common Words Among Other Presidential Inauguration Speeches")
    plt.xlabel('President')
    plt.ylabel('Words')
    plt.show()
'''